In [12]:
import re
import pandas as pd
from requests import get
url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2018-01-01,2018-12-31&start=51&ref_=adv_nxt'
response = get(url)


In [13]:
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')
movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-advanced')

In [14]:
df = pd.DataFrame(columns = ['Film title', 'Imdb 2018 list number', 'Year released', 'Runtime', 'Imdb Score', 'Metascore', 'Genres', 'MPAA rating', 'Director(s)', 'Actor(s)', 'Votes', 'Gross'])

In [15]:
def names(container):
    name_list = []
    for elem in container(text=re.compile(r'Director')):
        if elem:
            name_list = elem.parent.text

            names = name_list.split('\n')
            names = [i for i in names if i] 
        
        
            names_s = []
            for name in names: 
                name_s = name.strip()
                name_s = name_s.strip(",")
                names_s.append(name_s)  
                names_s = [i for i in names_s if "|" not in i] 

            star_ind = names_s.index('Stars:')
            if star_ind:
                stars = names_s[star_ind + 1:len(names_s)]
            
                if names_s[0][0:8] == 'Director':
                    director_s = names_s[1:star_ind ]

    
        
    return director_s, stars    



In [16]:
def vote_gross(container):
    gross_tot = None
    votes = container.find('p', class_ = 'sort-num_votes-visible')
    if votes:
        votes = votes.text
        votes = votes.replace('\n', ' ')
        votes = votes.replace('|', '')
        votes = votes.strip(" ")
        votes = votes.split(" ")
        votes = [i for i in votes if i]
        if votes[0] == 'Votes:':
                vote_s = votes[1]
        if len(votes) > 2:
            if votes[2] == 'Gross:':
                gross_tot = votes[3]
         
    return  vote_s, gross_tot  

In [26]:

for container in movie_containers:
    data = []
    title = container.h3.a
    if title:
        data.append(title.text)
    else:
        data.append(None)
        
    list_number = container.find('span', class_ = 'lister-item-index unbold text-primary')
    if list_number:
        data.append(int(list_number.text.strip(".")))   
    else:
        data.append(None)
    
    year = container.h3.find('span', class_ = 'lister-item-year text-muted unbold')
    if year:
        
        data.append(year.text.strip("()"))
    else:
        data.append(None)
                    
    runtime = container.find('span', class_ = 'runtime')
    if runtime:
        data.append(runtime.text.strip("()"))
    else:
        data.append(None)
                    
    imdb_sc = container.strong
    if imdb_sc:
        data.append(float(imdb_sc.text.strip("()")))
    else:
        data.append(None)
                    
    mscore = container.find('span', class_ = 'metascore favorable')
    if mscore:
        data.append(int(mscore.text))
    else:
        data.append(None)                
                    
    genres = container.find('span', class_ = 'genre')
    if genres:
        data.append(genres.text.strip("\n"))
    else:
        data.append(None)                
                    
    MPAA_rating = container.find('span', class_ = 'certificate')
    if MPAA_rating:
        data.append(MPAA_rating.text)
    else:
        data.append(None)    
    
    
    director, star = names(container)
    if director:
        data.append(director)
    else:
        data.append(None)
                    
    if star:
        data.append(star)
    else:
        data.append(None)                
                    
    
    votes, gross = vote_gross(container)
                    
    if votes:
        data.append(votes)            
    else:
        data.append(None)
                    
    if gross:
        data.append(gross)                
    else:
        data.append(None)
                    
    
    df2 =  pd.DataFrame([data], columns = ['Film title', 'Imdb 2018 list number', 'Year released', 'Runtime', 'Imdb Score', 'Metascore', 'Genres', 'MPAA rating', 'Director(s)', 'Actor(s)', 'Votes', 'Gross'])              
    df = df.append(df2)
   
    

In [1]:
def names(container):
    name_list = []
    for elem in container(text=re.compile(r'Director')):
        if elem:
            name_list = elem.parent.text

            names = name_list.split('\n')
            names = [i for i in names if i] 
        
        
            names_s = []
            for name in names: 
                name_s = name.strip()
                name_s = name_s.strip(",")
                names_s.append(name_s)  
                names_s = [i for i in names_s if "|" not in i] 

            if names_s[0][0:8] == 'Director':
                    director = names_s[1]
            if 'Stars:' in names_s:
                star_ind = names_s.index('Stars:')
                if star_ind:
                    star = names_s[star_ind + 1]
            
                

    
        
    return director, star    



In [25]:
def vote_gross(container):
    gross_tot = None
    votes = container.find('p', class_ = 'sort-num_votes-visible')
    if votes:
        votes = votes.text
        votes = votes.replace('\n', ' ')
        votes = votes.replace('|', '')
        votes = votes.strip(" ")
        votes = votes.split(" ")
        votes = [i for i in votes if i]
        if votes[0] == 'Votes:':
                vote_s = int(votes[1].replace(',', ''))
        if len(votes) > 2:
            if votes[2] == 'Gross:':
                gross_tot = votes[3]
         
    return  vote_s, gross_tot   
       

In [27]:
df.tail()

,Film title,Imdb 2018 list number,Year released,Runtime,Imdb Score,Metascore,Genres,MPAA rating,Director(s),Actor(s),Votes,Gross
0,Nekrotronic,96,2018,99 min,5.5,None,"Comedy, Horror, Sci-Fi",R,Kiah Roache-Turner,Ben O'Toole,698,None
0,To All the Boys I've Loved Before,97,2018,99 min,7.2,64,"Drama, Romance",TV-14,Susan Johnson,Lana Condor,67090,None
0,Replicas,98,2018,107 min,5.5,None,"Drama, Sci-Fi, Thriller",PG-13,Jeffrey Nachmanoff,Keanu Reeves,21335,$4.05M
0,Fifty Shades Freed,99,2018,105 min,4.5,None,"Drama, Romance, Thriller",R,James Foley,Dakota Johnson,45707,$100.41M
0,Nightmare Cinema,100,2018,119 min,5.4,None,Horror,R,Alejandro Brugués,Mickey Rourke,1377,None


In [7]:
x = '2,900,999'
int(x.replace(',', ''))

2900999

In [8]:
x = '2018''

SyntaxError: EOL while scanning string literal (<ipython-input-8-d0cc177e0f72>, line 1)